<a href="https://colab.research.google.com/github/skytecat/CatOrDog/blob/main/cat_and_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Импорты

In [2]:
import pandas as pd
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import kagglehub
!pip install tensorflow -q

import warnings
warnings.filterwarnings('ignore', module='PIL.TiffImagePlugin')

In [3]:
# Путь к датасету
path = kagglehub.dataset_download("bhavikjikadara/dog-and-cat-classification-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/dog-and-cat-classification-dataset


In [4]:
cat_folder = os.path.join(path, 'PetImages', 'Cat')
dog_folder = os.path.join(path, 'PetImages', 'Dog')

## Предварительный анализ датасета

In [4]:
def check_images_shapes(folder):
    """Проверка форматов изображений с обработкой ошибок"""
    channels = {}
    sizes = []
    errors = 0
    processed = 0

    print(f"Загрузка из папки: {os.path.basename(folder)}")

    for filename in os.listdir(folder):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            try:
                img_path = os.path.join(folder, filename)

                # Открываем изображение
                img = Image.open(img_path)

                # Проверяем, можно ли прочитать полностью
                img.verify()  # Проверяет целостность

                # Если verify прошел, открываем заново (он закрывает файл)
                img = Image.open(img_path)
                original_width, original_height = img.size
                sizes.append((original_width, original_height))

                img_array = np.array(img)

                # Считаем статистику
                shape = img_array.shape
                if len(shape) == 2:
                  channels[1] = channels.get(1, 0) + 1
                else:
                  channels[shape[2]] = channels.get(shape[2], 0) + 1
                processed += 1

                # Показываем прогресс каждые 1000 файлов
                if processed % 1000 == 0:
                    print(f"  Обработано: {processed} файлов...")

            except Exception as e:
                errors += 1
                if errors <= 10:
                  print(f"  Ошибка с {filename}: {type(e).__name__}")

    print(f"\n  Всего обработано: {processed} файлов")
    print(f"  Ошибок: {errors}")

    return channels, sizes, processed, errors

def analyze_sizes(sizes, class_name):
    """Анализ размеров изображений"""
    if not sizes:
        print(f"Нет данных для анализа {class_name}")
        return

    widths = [s[0] for s in sizes]
    heights = [s[1] for s in sizes]
    aspect_ratios = [w/h for w, h in sizes]

    print(f"\nАНАЛИЗ РАЗМЕРОВ ИЗОБРАЖЕНИЙ {class_name.upper()}:")
    print(f"  Ширина: от {min(widths)} до {max(widths)} (средняя: {np.mean(widths):.0f})")
    print(f"  Высота: от {min(heights)} до {max(heights)} (средняя: {np.mean(heights):.0f})")
    print(f"  Соотношение сторон: от {min(aspect_ratios):.2f} до {max(aspect_ratios):.2f}")

    print(f"\n  Классификация по размерам:")
    print(f"    Очень маленькие (<32px): {sum(1 for w,h in sizes if w<32 or h<32)}")
    print(f"    Маленькие (32-64px): {sum(1 for w,h in sizes if (w>=32 and w<64) or (h>=32 and h<64))}")
    print(f"    Средние (64-128px): {sum(1 for w,h in sizes if (w>=64 and w<128) or (h>=64 and h<128))}")
    print(f"    Большие (128-256px): {sum(1 for w,h in sizes if (w>=128 and w<256) or (h>=128 and h<256))}")
    print(f"    Очень большие (256-512px): {sum(1 for w,h in sizes if (w>=256 and w<512) or (h>=256 and h<512))}")
    print(f"    Огромные (>512px): {sum(1 for w,h in sizes if w>512 or h>512)}")

    print(f"\n  Классификация по соотношениям сторон:")
    print(f"    Очень узкие (<0.5): {sum(1 for r in aspect_ratios if r < 0.5)}")
    print(f"    Узкие (0.5-0.8): {sum(1 for r in aspect_ratios if r >= 0.5 and r < 0.8)}")
    print(f"    Квадратные (0.8-1.2): {sum(1 for w,h in sizes if w==h or (min(w,h)/max(w,h) >= 0.8 and min(w,h)/max(w,h) <= 1.2))}")
    print(f"    Широкие (1.2-2.0): {sum(1 for r in aspect_ratios if r >= 1.2 and r < 2.0)}")
    print(f"    Очень широкие (>2.0): {sum(1 for r in aspect_ratios if r >= 2.0)}")

cat_channels, cat_sizes, cat_processed, cat_errors = check_images_shapes(
    '/kaggle/input/dog-and-cat-classification-dataset/PetImages/Cat'
)

print("\n" + "="*50)

dog_channels, dog_sizes, dog_processed, dog_errors = check_images_shapes(
    '/kaggle/input/dog-and-cat-classification-dataset/PetImages/Dog'
)

Загрузка из папки: Cat
  Обработано: 1000 файлов...
  Обработано: 2000 файлов...
  Обработано: 3000 файлов...
  Обработано: 4000 файлов...
  Обработано: 5000 файлов...
  Обработано: 6000 файлов...
  Обработано: 7000 файлов...
  Обработано: 8000 файлов...
  Обработано: 9000 файлов...
  Обработано: 10000 файлов...
  Обработано: 11000 файлов...
  Обработано: 12000 файлов...

  Всего обработано: 12499 файлов
  Ошибок: 0

Загрузка из папки: Dog
  Обработано: 1000 файлов...
  Обработано: 2000 файлов...
  Обработано: 3000 файлов...
  Обработано: 4000 файлов...
  Обработано: 5000 файлов...
  Обработано: 6000 файлов...
  Обработано: 7000 файлов...
  Обработано: 8000 файлов...
  Обработано: 9000 файлов...
  Обработано: 10000 файлов...
  Обработано: 11000 файлов...
  Обработано: 12000 файлов...

  Всего обработано: 12499 файлов
  Ошибок: 0


In [5]:
print("\nАНАЛИЗ ФОРМАТОВ ИЗОБРАЖЕНИЙ:")
print("\nФорматы кошачьих изображений:")
print(cat_channels)
print("\nФорматы собачьих изображений:")
print(dog_channels)
print("\n" + "="*50)

# Анализ размеров изображений
analyze_sizes(cat_sizes, "кошек")
print("\n" + "="*50)
analyze_sizes(dog_sizes, "собак")


АНАЛИЗ ФОРМАТОВ ИЗОБРАЖЕНИЙ:

Форматы кошачьих изображений:
{3: 12470, 1: 29}

Форматы собачьих изображений:
{3: 12461, 4: 5, 1: 33}


АНАЛИЗ РАЗМЕРОВ ИЗОБРАЖЕНИЙ КОШЕК:
  Ширина: от 4 до 500 (средняя: 411)
  Высота: от 4 до 500 (средняя: 357)
  Соотношение сторон: от 0.35 до 3.72

    Классификация по размерам:
    Очень маленькие (<32px): 1
    Маленькие (32-64px): 16
    Средние (64-128px): 258
    Большие (128-256px): 2325
    Очень большие (256-512px): 11353
    Огромные (>512px): 0

    Классификация по соотношениям сторон:
    Очень узкие (<0.5): 34
    Узкие (0.5-0.8): 1600
    Квадратные (0.8-1.2): 3923
    Широкие (1.2-2.0): 7209
    Очень широкие (>2.0): 109


АНАЛИЗ РАЗМЕРОВ ИЗОБРАЖЕНИЙ СОБАК:
  Ширина: от 42 до 500 (средняя: 398)
  Высота: от 33 до 500 (средняя: 365)
  Соотношение сторон: от 0.31 до 5.94

    Классификация по размерам:
    Очень маленькие (<32px): 0
    Маленькие (32-64px): 26
    Средние (64-128px): 242
    Большие (128-256px): 2240
    Очень большие (25

### Выводы по качеству данных

* У классов практически идентичное распределение
* Большинство изображений больших размеров (256-512px) ~ 90%
* Отличный баланс классов (50% кошек, 50% собак)
* У большинства изображений 3 канала (RGB) ~ 99%
* Изображений критически маленького размера или с экстремальными соотношениями сторон < 1%

### План предобработки





#### Удаление проблемных изображений
1. Критически маленькие изображения

- **Критерий**: ширина < 32px ИЛИ высота < 32px

- **Причина**: невозможно распознать объект, только шум

2. Изображения с экстремальными соотношениями сторон

- **Критерий**: соотношение сторон < 0.4 ИЛИ соотношение сторон > 3.0
- **Причина**: сильные искажения при resize, потеря информации


#### Унификация каналов
В нашем датасете есть следующие проблемы с разными каналами:
1. **Grayscale** изображения (1 канал)
- Проблема: модель ожидает 3 канала (RGB)
- Решение: конвертация в RGB путем дублирования канала
2. **RGBA** изображения (4 канала):
- Проблема: лишний альфа-канал (прозрачность)
- Решение: удаление альфа-канала, сохранение только RGB

#### Resize и padding изображений
Выбранный размер изображения после resize: **128×128** пикселей. Это баланс качества и производительности

Для приведения всех изображений к единому размеру 128×128 пикселей будем применять двухэтапный подход:

1. **Resize** с сохранением пропорций
Будет использоваться метод thumbnail() библиотеки PIL
- Цель: уменьшить изображение с сохранением оригинальных пропорций
- Преимущество: избегаем искажений объектов
- Пример: изображение 400×200 пропорционально уменьшается до 128×64
2. **Padding** для создания квадрата
После пропорционального уменьшения изображение помещается в центр черного квадрата


**Этот подход гарантирует:**

- Все изображения имеют размер 128×128 пикселей
- Сохраняются оригинальные пропорции объектов
- Отсутствуют искажения изображений
- Черные поля не мешают классификации, так как содержат только фон


## Загрузка и подготовка данных

In [5]:
def load_and_preprocess_dataset(folder, label, target_size, min_size):
    """
    Полная загрузка и предобработка изображений из папки

    Args:
        folder: путь к папке с изображениями
        target_size: целевой размер изображений (ширина, высота)
        min_size: минимальный размер изображения

    Returns:
        X: массив изображений формы (n_samples, height, width, 3)
        y: массив меток формы (n_samples)
    """

    def normalize_channels(img_array):
        """
        Приводит изображение к единому формату RGB (3 канала)
        """
        if len(img_array.shape) == 2:
            # Grayscale (1 канал) → RGB (3 канала)
            img_array = np.stack([img_array] * 3, axis=-1)

        elif len(img_array.shape) == 3:
            if img_array.shape[2] == 1:
                # (H, W, 1) → (H, W, 3)
                img_array = np.concatenate([img_array] * 3, axis=-1)

            elif img_array.shape[2] == 4:
                # RGBA (4 канала) → RGB (3 канала)
                img_array = img_array[:, :, :3]

            elif img_array.shape[2] != 3:
                # Неподдерживаемый формат
                raise ValueError(f"Неподдерживаемое количество каналов: {img_array.shape[2]}")

        return img_array

    # def smart_resize_with_padding(img, target_size):
    #     """
    #     Resize изображения с сохранением пропорций через padding
    #     """
    #     # Уменьшаем с сохранением пропорций (thumbnail сохраняет пропорции)
    #     img_copy = img.copy()
    #     img_copy.thumbnail(target_size, Image.Resampling.LANCZOS)

    #     # Создаем квадратный холст
    #     new_img = Image.new('RGB', target_size, (0, 0, 0))  # Черный фон

    #     # Центрируем изображение
    #     x = (target_size[0] - img_copy.size[0]) // 2
    #     y = (target_size[1] - img_copy.size[1]) // 2
    #     new_img.paste(img_copy, (x, y))

    #     return new_img

    def smart_resize_with_padding(img, target_size):
      """
      Resize изображения с сохранением пропорций через padding
      """
      try:
          # Уменьшаем с сохранением пропорций
          img_copy = img.copy()
          img_copy.thumbnail(target_size, Image.Resampling.LANCZOS)

          # Создаем квадратный холст нужного размера
          new_img = Image.new('RGB', target_size, (0, 0, 0))  # Черный фон

          # Центрируем изображение
          x = (target_size[0] - img_copy.size[0]) // 2
          y = (target_size[1] - img_copy.size[1]) // 2
          new_img.paste(img_copy, (x, y))

          # Проверяем финальный размер
          if new_img.size != target_size:
              # Принудительно изменяем до нужного размера если что-то пошло не так
              new_img = new_img.resize(target_size, Image.Resampling.LANCZOS)

          return new_img
      except Exception as e:
          print(f"Ошибка при resize: {e}")
          # Возвращаем стандартное изображение нужного размера
          return Image.new('RGB', target_size, (128, 128, 128))  # Серый фон

    def should_keep_image(width, height, aspect_ratio, min_size=32):
        """
        Определяет, стоит ли оставлять изображение
        """
        # Минимальный размер
        if min(width, height) < min_size:
            return False

        # Экстремальные соотношения сторон
        if aspect_ratio < 0.4 or aspect_ratio > 3.0:
            return False

        return True

    images = []
    labels = []
    stats = {
        'processed': 0,
        'deleted': 0,
        'corrupted': 0
    }

    print(f"Загружаем изображения из папки {os.path.basename(folder)}")

    for filename in os.listdir(folder):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            try:
                img_path = os.path.join(folder, filename)

                # Открываем изображение
                img = Image.open(img_path)
                width, height = img.size
                aspect_ratio = width / height

                # Проверяем, стоит ли оставлять изображение
                if not should_keep_image(width, height, aspect_ratio, min_size):
                    stats['deleted'] += 1
                    continue

                # Применяем smart resize с padding
                img_resized = smart_resize_with_padding(img, target_size)
                # img_resized = img.resize(target_size)

                # Конвертируем в массив и нормализуем
                img_array = np.array(img_resized) / 255.0

                # Унифицируем каналы
                if len(img_array.shape) == 2 or (len(img_array.shape) == 3 and img_array.shape[2] != 3):
                  img_array = normalize_channels(img_array)

                # Проверяем финальный формат
                if img_array.shape == (*target_size, 3):
                    images.append(img_array)
                    labels.append(label)
                    stats['processed'] += 1
                else:
                    stats['corrupted'] += 1

                # Показываем прогресс
                if stats['processed'] % 1000 == 0:
                    print(f"  Обработано: {stats['processed']} изображений...")

            except Exception as e:
                stats['corrupted'] += 1
                if stats['corrupted'] <= 10:
                    print(f"  Ошибка при обработке {filename}: {type(e).__name__}")

    print(f"\n  Загрузка файлов завершена:")
    print(f"  Загружено: {stats['processed']} изображений")
    print(f"  Отфильтровано: {stats['deleted'] + stats['corrupted']} изображений")
    print("="*60)

    return images, labels, stats

Возникают проблемы с изображениями 128x128

In [6]:
    # Загружаем изображения обоих классов
    min_size = 32
    # target_size=(128, 128)
    target_size=(64, 64)

    cat_images, cat_labels, cat_stats = load_and_preprocess_dataset(
        cat_folder, 0, target_size, min_size
    )

    # dog_images, dog_labels, dog_stats = load_and_preprocess_dataset(
    #     dog_folder, 1, target_size, min_size
    # )


Загружаем изображения из папки Cat
  Обработано: 1000 изображений...
  Обработано: 2000 изображений...
  Обработано: 3000 изображений...
  Обработано: 4000 изображений...
  Обработано: 5000 изображений...
  Обработано: 6000 изображений...
  Обработано: 7000 изображений...
  Обработано: 8000 изображений...
  Обработано: 9000 изображений...
  Обработано: 10000 изображений...
  Обработано: 11000 изображений...
  Обработано: 12000 изображений...

  Загрузка файлов завершена:
  Загружено: 12489 изображений
  Отфильтровано: 10 изображений


In [7]:
    # # Объединяем данные
    # X = np.array(cat_images + dog_images)
    # y = np.array(cat_labels + dog_labels)

    X = np.array(cat_images)


    # # Выводим статистику
    # print("\n" + "="*60)
    # print("СТАТИСТИКА ЗАГРУЗКИ")
    # print("="*60)
    # print(f"Кошки: {len(cat_images)} изображений")
    # # print(f"Собаки: {len(dog_images)} изображений")
    # # print(f"Всего: {len(X)} изображений")
    # print(f"Форма данных: {X.shape}")
    # print(f"Диапазон значений: {X.min():.3f} - {X.max():.3f}")

    # print(f"\nСтатистика кошек:")
    # for key, value in cat_stats.items():
    #     print(f"  {key}: {value}")

    # print(f"\nСтатистика собак:")
    # for key, value in dog_stats.items():
    #     print(f"  {key}: {value}")

In [ ]:
# def visualize_sample_images(X, y, n_samples=6):
#     """
#     Визуализация примеров изображений
#     """
#     plt.figure(figsize=(15, 6))

#     # Выбираем случайные изображения
#     indices = np.random.choice(len(X), min(n_samples, len(X)), replace=False)

#     for i, idx in enumerate(indices):
#         plt.subplot(2, 3, i + 1)
#         plt.imshow(X[idx])
#         plt.title(f"{'Кошка' if y[idx] == 0 else 'Собака'}")
#         plt.axis('off')

#     plt.suptitle('Примеры изображений после предобработки')
#     plt.tight_layout()
#     plt.show()

# def split_dataset(X, y, test_size=0.2, val_size=0.1, random_state=42):
#     """
#     Разделение датасета на train/validation/test
#     """
#     # Сначала делим на train+val и test
#     X_temp, X_test, y_temp, y_test = train_test_split(
#         X, y, test_size=test_size, random_state=random_state, stratify=y
#     )

#     # Затем делим train+val на train и validation
#     val_size_adjusted = val_size / (1 - test_size)  # Корректируем размер валидации
#     X_train, X_val, y_train, y_val = train_test_split(
#         X_temp, y_temp, test_size=val_size_adjusted, random_state=random_state, stratify=y_temp
#     )

#     print(f"Разделение датасета:")
#     print(f"  Обучение: {len(X_train)} изображений ({len(X_train)/len(X)*100:.1f}%)")
#     print(f"  Валидация: {len(X_val)} изображений ({len(X_val)/len(X)*100:.1f}%)")
#     print(f"  Тест: {len(X_test)} изображений ({len(X_test)/len(X)*100:.1f}%)")

#     # Проверяем баланс классов
#     print(f"\nБаланс классов:")
#     print(f"  Обучение - Кошки: {np.sum(y_train == 0)}, Собаки: {np.sum(y_train == 1)}")
#     print(f"  Валидация - Кошки: {np.sum(y_val == 0)}, Собаки: {np.sum(y_val == 1)}")
#     print(f"  Тест - Кошки: {np.sum(y_test == 0)}, Собаки: {np.sum(y_test == 1)}")

#     return X_train, X_val, X_test, y_train, y_val, y_test

# # Использование:
# if __name__ == "__main__":
#     # Пути к папкам (измени на свои)
#     cat_folder = '/kaggle/input/dog-and-cat-classification-dataset/PetImages/Cat'
#     dog_folder = '/kaggle/input/dog-and-cat-classification-dataset/PetImages/Dog'

#     # Загружаем и предобрабатываем данные
#     print("Начинаем загрузку и предобработку данных...")
#     X, y = load_and_preprocess_dataset(
#         cat_folder=cat_folder,
#         dog_folder=dog_folder,
#         target_size=(128, 128),
#         min_size=32
#     )

#     # Визуализируем примеры
#     print("\nПоказываем примеры изображений...")
#     visualize_sample_images(X, y)

#     # Разделяем на выборки
#     print("\nРазделяем датасет...")
#     X_train, X_val, X_test, y_train, y_val, y_test = split_dataset(X, y)

#     print(f"\n✅ Готово! Данные загружены и готовы для обучения.")
#     print(f"   Форма обучающей выборки: {X_train.shape}")
#     print(f"   Форма валидационной выборки: {X_val.shape}")
#     print(f"   Форма тестовой выборки: {X_test.shape}")

In [17]:
def load_images(folder, label, img_size=(128, 128)):
    images = []
    labels = []
    for filename in os.listdir(folder):
      if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        try:
            img = Image.open(os.path.join(folder, filename))
            img = img.resize(img_size)
            img_array = np.array(img) / 255.0  # нормализация

            # Grayscale изображения
            if len(img_array.shape) == 2:
                img_array = np.stack([img_array]*3, axis=-1)
            elif img_array.shape[2] == 4:
                # RGBA -> RGB: убираем альфа-канал
                 img_array = img_array[:,:,:3]

            images.append(img_array)
            labels.append(label)
        except Exception as e:
            print(f"Ошибка: {filename} - {e}")
    return images, labels

cat_images, cat_labels = load_images('/kaggle/input/dog-and-cat-classification-dataset/PetImages/Cat', 0)
dog_images, dog_labels = load_images('/kaggle/input/dog-and-cat-classification-dataset/PetImages/Dog', 1)

/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


In [3]:
X = np.array(cat_images + dog_images)
y = np.array(cat_labels + dog_labels)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Обучение моделей
1. Базовая модель
2. Модель с регуляризацией
3. Модель с аугментацией
4. Финальная

### Модель 1: Базовая архитектура

In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

# Создаем правильную архитектуру
model = models.Sequential([
    layers.Input(shape=(64,64,3)),

    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Компилируем
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # Теперь следим за loss
    patience=5,
    restore_best_weights=True
)

# Обучаем
history = model.fit(
    X_train, y_train,
    epochs=30,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping]
)

Epoch 1/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 132s 208ms/step - accuracy: 0.5184 - loss: 0.6917 - val_accuracy: 0.6670 - val_loss: 0.6377
Epoch 2/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 127s 204ms/step - accuracy: 0.6728 - loss: 0.6139 - val_accuracy: 0.7374 - val_loss: 0.5286
Epoch 3/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 141s 203ms/step - accuracy: 0.7252 - loss: 0.5490 - val_accuracy: 0.7536 - val_loss: 0.4943
Epoch 4/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 131s 209ms/step - accuracy: 0.7606 - loss: 0.4981 - val_accuracy: 0.7846 - val_loss: 0.4578
Epoch 5/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 140s 205ms/step - accuracy: 0.7902 - loss: 0.4585 - val_accuracy: 0.8042 - val_loss: 0.4254
Epoch 6/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 128s 205ms/step - accuracy: 0.8003 - loss: 0.4356 - val_accuracy: 0.8166 - val_loss: 0.4039
Epoch 7/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 127s 204ms/step - accuracy: 0.8176 - loss: 0.4038 - val_accuracy: 0.8156 - val_loss: 0.3957
Epoch 8/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 144s 208ms/step - accuracy: 0.8243 -

KeyboardInterrupt: 

In [8]:
loss, accuracy = model.evaluate(X_test, y_test)

157/157 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step - accuracy: 0.8434 - loss: 0.3617
